In [1]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
import re
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from nltk.tokenize import sent_tokenize
import random

def remove_bracketed_sections(text):
    text = re.sub(r'\[.?\]', '', text)
    table = str.maketrans({';': '', '"': '', ' ': ' '})
    return text.translate(table)

def preprocess_lyrics(text):
    text = re.sub(r'\n{2,}', '\n', text)
    text = re.sub(r'\(.?\)', '', text)
    text = re.sub(r'\s\n\s', '\n', text)
    return text.strip()

df = pd.read_csv('slava_kpss_lyrics.csv')
df['lyrics'] = df['lyrics'].apply(remove_bracketed_sections)
df['lyrics'] = df['lyrics'].apply(preprocess_lyrics)

all_sentences = []
for s in df['lyrics'].dropna():
    all_sentences += sent_tokenize(s, language='russian')

all_songs = "\n".join(all_sentences)
with open("all_songs.txt", "w", encoding="utf-8") as f:
    f.write(all_songs)





In [3]:
class MarkovChain:
    def __init__(self, order=2):
        self.order = order
        self.chain = defaultdict(Counter)
        self.words = []

    def train(self, text_data):
        for sentence in text_data:
            words = sentence.split()
            if len(words) < self.order + 1:
                continue

            words = ['<START>'] * self.order + words + ['<END>']
            self.words.extend(words)

            for i in range(len(words) - self.order):
                key = tuple(words[i:i + self.order])
                next_word = words[i + self.order]
                self.chain[key][next_word] += 1

    def get_next_word(self, context, temperature=1.0):
        word_counts = self.chain[context]
        words = list(word_counts.keys())
        counts = list(word_counts.values())
        if not words:
            return '<END>'
        counts = np.array(counts, dtype=float)
        log_probs = np.log(counts + 1e-8) / temperature
        probs = np.exp(log_probs - np.max(log_probs))
        probs = probs / np.sum(probs)
        return np.random.choice(words, p=probs)


In [4]:
markov_model = MarkovChain(order=2)
markov_model.train(all_sentences)

In [5]:
def generate_text_markov(model, start_string="", temperature=0.5, max_length=100):
    if start_string.strip():
        # Если есть начальная строка, использую ее
        words = start_string.strip().split()
        generated_words = words.copy()
        if len(words) >= model.order:
            context = tuple(words[-model.order:])
        else:
            # Если слов меньше чем order, дополняем START токенами
            context = tuple(['<START>'] * (model.order - len(words)) + words)
    else:
        context = tuple(['<START>'] * model.order)
        generated_words = []

    for _ in range(max_length):
        next_word = model.get_next_word(context, temperature=temperature)
        if next_word == '<END>':
            break
        if next_word != '<START>':
            generated_words.append(next_word)
        context = context[1:] + (next_word,)

    return ' '.join(generated_words)

In [8]:
start_texts = [
    "На пятки наступает",
    "Я не тот, кем был раньше",
    "Это всё просто мельтешение"
]
for start_text in start_texts:
    print(f"\nНачальный текст: '{start_text}'")
    generated_text = generate_text_markov(
        markov_model,
        start_text,
        temperature=0.5,
        max_length=50
    )

    print(generated_text)


Начальный текст: 'На пятки наступает'
На пятки наступает духовная нищанка Сыграй со мной в пятнашки, я — весёлая Каштанка Гоняюсь по двору за собственным хвостом И так покуда не помру, и, видно, в этом весь прикол [Припев] А дедушка ждёт, а дедушка плачет Но для ЖЭКа его слёзы мало чего значат [Куплет] Он думал, знает он про тебя

Начальный текст: 'Я не тот, кем был раньше'
Я не тот, кем был раньше По мне не суйся к нам придёт, тот охуеет от того, что ты вкусняшка Ты собака, я упряжка, спускай вперёд Годики своё берут, скажешь: Скоро наберу Как крестьяне заберу, мы ведём войну Мой крю строго на говне Говорим расизму: «Нет» Защищал Россию дед От таких как я закрыты Так что

Начальный текст: 'Это всё просто мельтешение'
Это всё просто мельтешение жизни Отмечаю вещи и мысли серьезные А только белый синий красный, и были правы все фантасты Айн Рэнд и Голдинг, Хаксли, что социум как антипод прекрасного и справедливого Похорони меня, где ветер в ивах нежно шепчет не буди Не будь самим собой

In [ ]:
import pickle

def save_markov_model(model, filepath='markov_model.pkl'):
    with open(filepath, 'wb') as f:
        pickle.dump(model, f)

def load_markov_model(filepath='markov_model.pkl'):
    with open(filepath, 'rb') as f:
        model = pickle.load(f)
    return model
save_markov_model(markov_model)